In [158]:
!pip install graphdatascience

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 48.8 MB/s eta 0:00:00


In [159]:
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd
import numpy as np

In [ ]:
!python main.py --dataset emails --nt 1000 --nq 1000 --epochs 20 --save-split --recall --save-embed --save-model

In [218]:
email_df = pd.read_csv("data/emails", header=None)
email_df.columns=['email']

In [219]:
email_df[email_df['email']=='shawn34@gmail.com']

,email
965,shawn34@gmail.com


In [191]:
!ls model/808/cnn/emails/nt1000_nq1000

base_idx.npy	  embedding_xt.npy  query_idx.npy   train_idx.npy
embedding_xb.npy  model.torch	    query_knn.npy   train_knn.npy
embedding_xq.npy  query_dist.npy    train_dist.npy


In [244]:
base_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/embedding_xb.npy')
train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/embedding_xt.npy')
query_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/embedding_xq.npy')

In [245]:
embedding_df = pd.concat([pd.DataFrame(base_embedding), pd.DataFrame(train_embedding), pd.DataFrame(query_embedding)])

In [247]:
base_index = train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/base_idx.npy')
train_index = train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/train_idx.npy')
query_index = train_embedding = np.load('model/808/cnn/emails/nt1000_nq1000/query_idx.npy')

In [248]:
embedding_index = pd.concat([pd.Series(base_index), pd.Series(train_index), pd.Series(query_index)])

In [249]:
embedding_df.index = embedding_index

In [250]:
embedding_df['embedding'] = embedding_df.apply(lambda row: row.tolist(), axis=1)

In [252]:
embedding_df = embedding_df['embedding'].copy()

In [253]:
embedding_df = email_df.merge(embedding_df, left_index=True, right_index=True)

In [254]:
embedding_df.head()

,email,embedding
0,johnbrooks@hotmail.com,"[0.07029952108860016, 0.004608599003404379, -0..."
1,kellyalejandra@potter.net,"[0.006088620983064175, -0.00497810123488307, 0..."
2,teresa28@roach.com,"[0.0385185070335865, -0.037428926676511765, 0...."
3,roweamy@martinez-douglas.com,"[0.10387729853391647, 0.023334773257374763, 0...."
4,sjennings@rivera.com,"[0.00400144886225462, -0.00797479972243309, 0...."


In [210]:
neo4j_password = getpass("neo4j password")

neo4j password ········


In [211]:
gds = GraphDataScience("neo4j+s://8b188457.databases.neo4j.io", auth=("neo4j", neo4j_password))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


In [256]:
gds.run_cypher("""CREATE CONSTRAINT email_node_key FOR (e:Email) REQUIRE e.address IS NODE KEY""")

""


In [258]:
gds.run_cypher("""
UNWIND $data AS row
CALL {
    WITH row
    MERGE (e:Email {address:row['email']})
    WITH row, e
    CALL db.create.setNodeVectorProperty(e, 'editEmbedding', row['embedding']) 
} IN CONCURRENT TRANSACTIONS OF 10000 rows""",
              {"data": embedding_df.to_dict("records")})

""
